In [1]:
!pip install -q -U transformers
!pip install -q -U accelerate
!pip install -q -U bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 46.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 46.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 62.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 66.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [2]:
from pathlib import Path
import pandas as pd

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch
import os
from tqdm import tqdm
import random

## Read Data

In [3]:
# Run this cell if you are using Google Colaboratory

# from google.colab import drive
# drive.mount('/content/drive')

# BASE_DIR = Path("/content/drive/MyDrive/{your_path}")
# DATA_DIR = BASE_DIR.joinpath("data")   # Please upload the data_quality_estim.csv dataset to the data folder 

# FILE_NAME = "data_topic_name_distrib.csv"

# FILE = DATA_DIR.joinpath(FILE_NAME)

Mounted at /content/drive


In [3]:
BASE_DIR = Path('Topic_Generation_Mistral_7B_v0_1_4bit.ipynb').resolve().parent.parent

DATA_DIR_NAME = 'data/datasets'  # Local directory

DATA_DIR = BASE_DIR.joinpath(DATA_DIR_NAME)
FILE_NAME_QUAL = 'data_topic_name_distrib.csv'

FILE = DATA_DIR.joinpath(FILE_NAME_QUAL)

In [4]:
data = pd.read_csv(FILE)
data.head(10)

,INPUT:wordset,OUTPUT:quality,OUTPUT:topic_name,topic_name_num
0,able amaze fill stop glad daily fantastic move...,good,feelings,1
1,absolute mission church nasa mars earth truth ...,good,astrology,1
2,absolute mission church nasa mars earth truth ...,good,nasa,1
3,absolute mission church nasa mars earth truth ...,good,space,1
4,absolute mission church nasa mars earth truth ...,good,space mission,1
5,absolutely fast white rice service noodle terr...,good,customer experience,1
6,absolutely fast white super service customer r...,good,express expedition,1
7,absolutely fast white super service customer r...,good,online shopping,2
8,absolutely fast white super service customer r...,good,review,1
9,absolutely grain there's process terrible expe...,good,comment,1


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5766 entries, 0 to 5765
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   INPUT:wordset      5766 non-null   object
 1   OUTPUT:quality     5766 non-null   object
 2   OUTPUT:topic_name  5766 non-null   object
 3   topic_name_num     5766 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 180.3+ KB


In [6]:
data_text = list(set(data["INPUT:wordset"]))

In [7]:
!nvidia-smi

Wed Apr  3 19:30:39 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [8]:
# Check if cuda is available
assert torch.cuda.is_available(), "Cuda is not available, you cannot use the quantized version of the Mistral model without a GPU!"

In [9]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

In [10]:
model_name = "mistralai/Mistral-7B-Instruct-v0.1"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config,
    )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

## Pipeline of the Inference

In [11]:
def generate_answer(prompt, model, device, temp, n_token):
    encoded = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
    model_input = encoded
    model_input = model_input.to(device)
    generated_ids = model.generate(
        **model_input, do_sample=True,
        max_new_tokens=n_token,
        temperature=temp,
        num_return_sequences=1,
        pad_token_id=tokenizer.eos_token_id
        )
    decoded = tokenizer.batch_decode(generated_ids)

    return decoded

In [12]:
def extract_substring(input_string):
    index = input_string.find("[/INST]")
    if index != -1:
        return input_string[index + len("[/INST]"):]
    else:
        return ""

In [41]:
def few_shot_pipeline(
        instruction,
        sample,
        model,
        device='cpu',
        temp=0.1,
        n_token=100,
        print_system_prompt=False,
        save_system_prompt=False
        ):

    # Construct prompt
    prompt = f"""<s> [INST] {instruction} \nText: {sample}\nTopic: [/INST]"""
    if print_system_prompt:
        print(prompt)
    # Generate answer
    answer = generate_answer(prompt, model, device, temp, n_token)
    # Extract the result
    if save_system_prompt:
        result = answer[0]
    else:
        result = extract_substring(answer[0])
    return result.replace("\n", " ")


## Prompt Engineering

#### Type I. 3-shot Chain-of-Thought

In [42]:
prompt = """<s> [INST] Name the common topic with one or few words.
Text: absolute mission church nasa mars earth truth lunar shuttle doug telescope symbol planetary libxmu launch
Topic: space
Text: anonymous public private news crime server religion carry pretty bill cview internet paul hold thought
Topic: news
Text: apple mouse robert fast processor speed move slow clock week pretty microsoft music chicago jason
Topic: [/INST]"""

answer = generate_answer(prompt, model, 'cuda', temp=0.2, n_token=100)

print(answer[0])


<s>  [INST] Name the common topic with one or few words.
Text: absolute mission church nasa mars earth truth lunar shuttle doug telescope symbol planetary libxmu launch
Topic: space
Text: anonymous public private news crime server religion carry pretty bill cview internet paul hold thought
Topic: news
Text: apple mouse robert fast processor speed move slow clock week pretty microsoft music chicago jason
Topic: [/INST] Technology</s>


Conclusion: The model provides an answer in the required format, this prompt can be used for further research

#### Type II. 3-shot Chain-of-Thought

In [18]:
prompt = """<s> [INST] Name the common topic with one or few words. Please, make sure you to only return the topic and nothing more.
Text: absolute mission church nasa mars earth truth lunar shuttle doug telescope symbol planetary libxmu launch
Topic: space
Text: anonymous public private news crime server religion carry pretty bill cview internet paul hold thought
Topic: news
Text: apple mouse robert fast processor speed move slow clock week pretty microsoft music chicago jason
Topic: [/INST]"""

answer = generate_answer(prompt, model, 'cuda', temp=0.2, n_token=100)

print(answer[0])


<s>  [INST] Name the common topic with one or few words. Please, make sure you to only return the topic and nothing more.
Text: absolute mission church nasa mars earth truth lunar shuttle doug telescope symbol planetary libxmu launch
Topic: space
Text: anonymous public private news crime server religion carry pretty bill cview internet paul hold thought
Topic: news
Text: apple mouse robert fast processor speed move slow clock week pretty microsoft music chicago jason
Topic: [/INST] space, news, technology</s>


Conclusion: The model provides an answer in the required format, this prompt can be used for further research

#### Type III. Chain-of-Thought

In [19]:
prompt = """<s> [INST] Name the common topic with one or few words.
Text: apple mouse robert fast processor speed move slow clock week pretty microsoft music chicago jason
Topic: [/INST]"""

answer = generate_answer(prompt, model, 'cuda', temp=0.2, n_token=100)

print(answer[0])

<s>  [INST] Name the common topic with one or few words.
Text: apple mouse robert fast processor speed move slow clock week pretty microsoft music chicago jason
Topic: [/INST] Technology, Computers, Software, Music, Chicago, Jason, Microsoft, Apple, Processor, Speed, Clock, Mouse.</s>


Conclusion: The examples were removed to avoid re-generation. At the beginning, the model tries to highlight the main topics that combine words from the set, but then simply ranks the words from the list

#### Type IV. Chain-of-Thought

In [20]:
prompt = """<s> [INST] Name the common topic with one or few words. Please, make sure you to only return the topic and nothing more.
Text: apple mouse robert fast processor speed move slow clock week pretty microsoft music chicago jason
Topic: [/INST]"""

answer = generate_answer(prompt, model, 'cuda', temp=0.2, n_token=100)

print(answer[0])

<s>  [INST] Name the common topic with one or few words. Please, make sure you to only return the topic and nothing more.
Text: apple mouse robert fast processor speed move slow clock week pretty microsoft music chicago jason
Topic: [/INST] apple, mouse, robert, fast, processor, speed, move, slow, clock, week, pretty, microsoft, music, chicago, jason</s>


Conclusion: Added clarification about the form of the response. The model outputs just a list of words

#### Type V. 3-shot Chain-of-Thought

In [21]:
prompt = """<s> [INST] Analyze the list of words and name the common topic with one or few words.

Text: absolute mission church nasa mars earth truth lunar shuttle doug telescope symbol planetary libxmu launch
Topic: space
Text: bar sweet crunchy yummy cinnamon granola yogurt crunch cereal raisin flake snack almond combination blueberry
Topic: sweet and sugar drink
Text: apple mouse robert fast processor speed move slow clock week pretty microsoft music chicago jason
Topic: [/INST]"""

answer = generate_answer(prompt, model, 'cuda', temp=0.2, n_token=100)

print(answer[0])

<s>  [INST] Analyze the list of words and name the common topic with one or few words.

Text: absolute mission church nasa mars earth truth lunar shuttle doug telescope symbol planetary libxmu launch
Topic: space
Text: bar sweet crunchy yummy cinnamon granola yogurt crunch cereal raisin flake snack almond combination blueberry
Topic: sweet and sugar drink
Text: apple mouse robert fast processor speed move slow clock week pretty microsoft music chicago jason
Topic: [/INST] The common topic with one or few words for the given text is "space" for the first text and "sweet and sugar drink" for the second text.</s>


Conclusion: The model repeats the topics for the "shots"

#### Type VI. 3-shot Chain-of-Thought

In [22]:
prompt = """<s> [INST] Analyze the list of words and name the common topic with one or few words. Please, make sure you to only return the topic and nothing more.

Text: absolute mission church nasa mars earth truth lunar shuttle doug telescope symbol planetary libxmu launch
Topic: space
Text: bar sweet crunchy yummy cinnamon granola yogurt crunch cereal raisin flake snack almond combination blueberry
Topic: sweet and sugar drink
Text: apple mouse robert fast processor speed move slow clock week pretty microsoft music chicago jason
Topic: [/INST]"""

answer = generate_answer(prompt, model, 'cuda', temp=0.2, n_token=100)

print(answer[0])

<s>  [INST] Analyze the list of words and name the common topic with one or few words. Please, make sure you to only return the topic and nothing more.

Text: absolute mission church nasa mars earth truth lunar shuttle doug telescope symbol planetary libxmu launch
Topic: space
Text: bar sweet crunchy yummy cinnamon granola yogurt crunch cereal raisin flake snack almond combination blueberry
Topic: sweet and sugar drink
Text: apple mouse robert fast processor speed move slow clock week pretty microsoft music chicago jason
Topic: [/INST] 1. space
2. sweet and sugar drink
3. [No clear topic found in this text.]</s>


Conclusion: The model repeats all the topics for the "shots", but cannot determine the topic for the proposed text

#### Type VII. Chain-of-Thought

In [23]:
prompt = """<s> [INST] Analyze the list of words and name the common topic with one or few words.

Text: apple mouse robert fast processor speed move slow clock week pretty microsoft music chicago jason
Topic:[/INST]"""

answer = generate_answer(prompt, model, 'cuda', temp=0.2, n_token=100)

print(answer[0])

<s>  [INST] Analyze the list of words and name the common topic with one or few words.

Text: apple mouse robert fast processor speed move slow clock week pretty microsoft music chicago jason
Topic:[/INST] Technology, Computers, Software, Internet, Music, Chicago, Microsoft, Processor, Speed, Clock, Robert, Jason, Mouse.</s>


Conclusion: The "shots" from the prompt have been removed. At the beginning, the model highlights topics that can combine words from the list, but then simply repeats the entire list

#### Type VII. Chain-of-Thought

In [25]:
prompt = """<s> [INST] Analyze the list of words and name the common topic with one or few words. Please, make sure you to only return the topic and nothing more.

Text: apple mouse robert fast processor speed move slow clock week pretty microsoft music chicago jason
Topic: [/INST]"""

answer = generate_answer(prompt, model, 'cuda', temp=0.2, n_token=100)

print(answer[0])

<s>  [INST] Analyze the list of words and name the common topic with one or few words. Please, make sure you to only return the topic and nothing more.

Text: apple mouse robert fast processor speed move slow clock week pretty microsoft music chicago jason
Topic: [/INST] Technology</s>


Conclusion: A clarification has been added in which form the response is expected from the model. This product can be used for further research

#### Type VIII. 3-shot Chain-of-Thought

In [27]:
prompt = """<s> [INST] You will receive a list of words and you should analyze it and name the common topic with one or few words.
Text: absolute mission church nasa mars earth truth lunar shuttle doug telescope symbol planetary libxmu launch
Topic: space
Text: anonymous public private news crime server religion carry pretty bill cview internet paul hold thought
Topic: news
Text: apple mouse robert fast processor speed move slow clock week pretty microsoft music chicago jason
Topic: [/INST]"""

answer = generate_answer(prompt, model, 'cuda', temp=0.2, n_token=100)

print(answer[0])

<s>  [INST] You will receive a list of words and you should analyze it and name the common topic with one or few words.
Text: absolute mission church nasa mars earth truth lunar shuttle doug telescope symbol planetary libxmu launch
Topic: space
Text: anonymous public private news crime server religion carry pretty bill cview internet paul hold thought
Topic: news
Text: apple mouse robert fast processor speed move slow clock week pretty microsoft music chicago jason
Topic: [/INST] Technology</s>


Conclusion: The model understands the task and gives the answer in the correct form. This prompt can be used for further analysis

#### Type IX. 3-shot Chain-of-Thought

In [28]:
prompt = """<s> [INST] You will receive a list of words and you should analyze it and name the common topic with one or few words. Please, make sure you to only return the topic and nothing more.
Text: absolute mission church nasa mars earth truth lunar shuttle doug telescope symbol planetary libxmu launch
Topic: space
Text: anonymous public private news crime server religion carry pretty bill cview internet paul hold thought
Topic: news
Text: apple mouse robert fast processor speed move slow clock week pretty microsoft music chicago jason
Topic: [/INST]"""

answer = generate_answer(prompt, model, 'cuda', temp=0.2, n_token=100)

print(answer[0])

<s>  [INST] You will receive a list of words and you should analyze it and name the common topic with one or few words. Please, make sure you to only return the topic and nothing more.
Text: absolute mission church nasa mars earth truth lunar shuttle doug telescope symbol planetary libxmu launch
Topic: space
Text: anonymous public private news crime server religion carry pretty bill cview internet paul hold thought
Topic: news
Text: apple mouse robert fast processor speed move slow clock week pretty microsoft music chicago jason
Topic: [/INST] Technology</s>


Conclusion: The model understands the task and gives the answer in the correct form (explanation added). This prompt can be used for further analysis

#### Type X. Chain-of-Thought

In [29]:
prompt = """<s> [INST] You will receive a list of words and you should analyze it and name the common topic with one or few words.
Text: apple mouse robert fast processor speed move slow clock week pretty microsoft music chicago jason
Topic: [/INST]"""

answer = generate_answer(prompt, model, 'cuda', temp=0.2, n_token=100)

print(answer[0])

<s>  [INST] You will receive a list of words and you should analyze it and name the common topic with one or few words.
Text: apple mouse robert fast processor speed move slow clock week pretty microsoft music chicago jason
Topic: [/INST] Technology, Computers, Software, Internet, Music, Chicago, Microsoft, Processor, Speed, Clock, Robert, Jason, Mouse.</s>


Conclusion: The examples have been removed. At the beginning, the model outputs topics that can combine all the words from the list, then the wordset is output from the prompt

#### Type XI. Chain-of-Thought

In [30]:
prompt = """<s> [INST] You will receive a list of words and you should analyze it and name the common topic with one or few words. Please, make sure you to only return the topic and nothing more.
Text: apple mouse robert fast processor speed move slow clock week pretty microsoft music chicago jason
Topic: [/INST]"""

answer = generate_answer(prompt, model, 'cuda', temp=0.2, n_token=100)

print(answer[0])

<s>  [INST] You will receive a list of words and you should analyze it and name the common topic with one or few words. Please, make sure you to only return the topic and nothing more.
Text: apple mouse robert fast processor speed move slow clock week pretty microsoft music chicago jason
Topic: [/INST] Technology</s>


Conclusion: A clarification has been added so that the model outputs only the required response. The model understands the task and gives the answer in the correct form. This prompt can be used for further analysis

#### Type XII. 3-shot Chain-of-Thought

In [33]:
prompt = """<s> [INST] You are a helpful, respectful and honest assistant for labeling topics. You will receive a list of words and you should analyze it and label list of words with one or few words.

Text: absolute mission church nasa mars earth truth lunar shuttle doug telescope symbol planetary libxmu launch
Label: space
Text: anonymous public private news crime server religion carry pretty bill cview internet paul hold thought
Label: news
Text: apple mouse robert fast processor speed move slow clock week pretty microsoft music chicago jason
Label: [/INST]"""

answer = generate_answer(prompt, model, 'cuda', temp=0.2, n_token=100)

print(answer[0])

<s>  [INST] You are a helpful, respectful and honest assistant for labeling topics. You will receive a list of words and you should analyze it and label list of words with one or few words.

Text: absolute mission church nasa mars earth truth lunar shuttle doug telescope symbol planetary libxmu launch
Label: space
Text: anonymous public private news crime server religion carry pretty bill cview internet paul hold thought
Label: news
Text: apple mouse robert fast processor speed move slow clock week pretty microsoft music chicago jason
Label: [/INST] Text: absolute mission church nasa mars earth truth lunar shuttle doug telescope symbol planetary libxmu launch
Label: space

Text: anonymous public private news crime server religion carry pretty bill cview internet paul hold thought
Label: news

Text: apple mouse robert fast processor speed move slow clock week pretty microsoft music chicago jason
Label: technology</s>


Conclusion: Added text specifying the role that the model should perform. The model understands what task it needs to complete, but repeats the examples. The answer is given in the correct form. This prompt can be used for further research, taking into account the post-processing of the results

#### Type XIII. 3-shot Chain-of-Thought

In [34]:
prompt = """<s> [INST] You are a helpful, respectful and honest assistant for labeling topics. You will receive a list of words and you should analyze it and label list of words with one or few words. Please, make sure you to only return the label and nothing more.

Text: absolute mission church nasa mars earth truth lunar shuttle doug telescope symbol planetary libxmu launch
Label: space
Text: anonymous public private news crime server religion carry pretty bill cview internet paul hold thought
Label: news
Text: apple mouse robert fast processor speed move slow clock week pretty microsoft music chicago jason
Label: [/INST]"""

answer = generate_answer(prompt, model, 'cuda', temp=0.2, n_token=100)

print(answer[0])

<s>  [INST] You are a helpful, respectful and honest assistant for labeling topics. You will receive a list of words and you should analyze it and label list of words with one or few words. Please, make sure you to only return the label and nothing more.

Text: absolute mission church nasa mars earth truth lunar shuttle doug telescope symbol planetary libxmu launch
Label: space
Text: anonymous public private news crime server religion carry pretty bill cview internet paul hold thought
Label: news
Text: apple mouse robert fast processor speed move slow clock week pretty microsoft music chicago jason
Label: [/INST] technology

Text: absolute mission church nasa mars earth truth lunar shuttle doug telescope symbol planetary libxmu launch
Label: space

Text: anonymous public private news crime server religion carry pretty bill cview internet paul hold thought
Label: news

Text: apple mouse robert fast processor speed move slow clock week pretty microsoft music chicago jason
Label: technolo

Conclusion: The model issues a response with the first token, then outputs examples from the prompt. This prompt can be used for further research, taking into account post-processing

#### Type XIV. Chain-of-Thought

In [35]:
prompt = """<s> [INST] You are a helpful, respectful and honest assistant for labeling topics. You will receive a list of words and you should analyze it and label list of words with one or few words.

Text: apple mouse robert fast processor speed move slow clock week pretty microsoft music chicago jason
Label: [/INST]"""

answer = generate_answer(prompt, model, 'cuda', temp=0.2, n_token=100)

print(answer[0])

<s>  [INST] You are a helpful, respectful and honest assistant for labeling topics. You will receive a list of words and you should analyze it and label list of words with one or few words.

Text: apple mouse robert fast processor speed move slow clock week pretty microsoft music chicago jason
Label: [/INST] apple: fruit
mouse: computer mouse
robert: name
fast: speed
processor: computer component
speed: computer performance
move: action
slow: speed
clock: timekeeping device
week: time unit
pretty: appearance
microsoft: computer company
music: sound
chicago: city
jason: name</s>


Conclusion: The examples have been removed. The model selects a topic for each word from the list

#### Type XV. Chain-of-Thought

In [36]:
prompt = """<s> [INST] You are the topic modeling assistant. You should extract the main theme that combines all the words from the list. Formulate it with a single word: apple mouse robert fast processor speed move slow clock week pretty microsoft music chicago jason [/INST]"""

answer = generate_answer(prompt, model, 'cuda', temp=0.2, n_token=100)

print(answer[0])

<s>  [INST] You are the topic modeling assistant. You should extract the main theme that combines all the words from the list. Formulate it with a single word: apple mouse robert fast processor speed move slow clock week pretty microsoft music chicago jason [/INST] The main theme that combines all the words from the list is "technology".</s>


Conclusion: The model correctly defines the topic. In this case, the model was given the task to generate only one example. This product can be used for further research

## Inference on the Entire Dataset

### Type III. Chain-of-Thought

In [43]:
INSTRUCTION = """Name the common topic with one or few words."""

SAMPLE = data_text[0]

res = few_shot_pipeline(INSTRUCTION, SAMPLE, model, device="cuda", print_system_prompt=True)

res

<s> [INST] Name the common topic with one or few words. 
Text: argument sense cannot statement conclusion therefore form assume position logic ence understanding imply fallacy argue
Topic: [/INST]


' Argumentation</s>'

In [ ]:
# Create file
RES_FILE_NAME = "data_topic_name_distrib_mistral_7b_4bit_type1_1.csv"
RES_FILE = DATA_DIR.joinpath(RES_FILE_NAME)
# Remove old file
if os.path.exists(RES_FILE):
    os.remove(RES_FILE)
with open(RES_FILE, 'a') as file:
    for i in tqdm(range(len(data_text))):
        SAMPLE = data_text[i]
        res_model = few_shot_pipeline(
            INSTRUCTION,
            SAMPLE,
            model,
            device="cuda",
            temp=0.2,
            n_token=100
        )
        file.write(f"{SAMPLE};{res_model}\n")


100%|██████████| 2363/2363 [58:53<00:00,  1.50s/it]


### Type VII. Chain-of-Thought

In [44]:
INSTRUCTION = """Analyze the list of words and name the common topic with one or few words. Please, make sure you to only return the topic and nothing more."""

SAMPLE = data_text[102]

res = few_shot_pipeline(INSTRUCTION, SAMPLE, model, device="cuda", print_system_prompt=True)

res

<s> [INST] Analyze the list of words and name the common topic with one or few words. Please, make sure you to only return the topic and nothing more. 
Text: pet vet feed ingredient wellness corn chicken liver meat diet filler science kibble kitten fructose
Topic: [/INST]


' The common topic in the list of words is "pet" and "vet".</s>'

In [ ]:
# Create file
RES_FILE_NAME = "data_topic_name_distrib_mistral_7b_4bit_type2_1.csv"
RES_FILE = DATA_DIR.joinpath(RES_FILE_NAME)
# Remove old file
if os.path.exists(RES_FILE):
    os.remove(RES_FILE)
for i in tqdm(range(len(data_text))):
    SAMPLE = data_text[i]
    res_model = few_shot_pipeline(
        INSTRUCTION,
        SAMPLE,
        model,
        device="cuda",
        temp=0.3,
        n_token=100
    )
    with open(RES_FILE, 'a') as file:
      file.write(f"{SAMPLE};{res_model}\n")


100%|██████████| 488/488 [23:41<00:00,  2.91s/it]


## Type X. Chain-of-Thought



In [45]:
INSTRUCTION = """You will receive a list of words and you should analyze it and name the common topic with one or few words."""

SAMPLE = data_text[180]

res = few_shot_pipeline(INSTRUCTION, SAMPLE, model, device="cuda", print_system_prompt=True)

res

<s> [INST] You will receive a list of words and you should analyze it and name the common topic with one or few words. 
Text: church truth absolute doug symbol borland bible scripture libxmu paradox catholic matter darren error certain
Topic: [/INST]


' The common topic in this list is "Religion/Spirituality" as many of the words are related to religious beliefs, practices, and figures.</s>'

In [ ]:
# Create file
RES_FILE_NAME = "data_topic_name_distrib_mistral_7b_4bit_type3_1.csv"
RES_FILE = DATA_DIR.joinpath(RES_FILE_NAME)
# Remove old file
if os.path.exists(RES_FILE):
    os.remove(RES_FILE)
with open(RES_FILE, 'a') as file:
    for i in tqdm(range(len(data_text))):
        SAMPLE = data_text[i]
        res_model = few_shot_pipeline(
            INSTRUCTION,
            SAMPLE,
            model,
            device="cuda",
            temp=0.3,
            n_token=100
        )
        file.write(f"{SAMPLE};{res_model}\n")


100%|██████████| 2363/2363 [1:41:28<00:00,  2.58s/it]


## Type XV. Chain-of-Thought


In [48]:
INSTRUCTION = """You are the topic modeling assistant. You should extract the main theme that combines all the words from the list. Formulate it with a single word:"""

SAMPLE = data_text[180]

res = few_shot_pipeline(INSTRUCTION, SAMPLE, model, device="cuda", print_system_prompt=True)

res

<s> [INST] You are the topic modeling assistant. You should extract the main theme that combines all the words from the list. Formulate it with a single word: 
Text: church truth absolute doug symbol borland bible scripture libxmu paradox catholic matter darren error certain
Topic: [/INST]


' The main theme that combines all the words from the list is "dogma".</s>'

In [ ]:
# Create file
RES_FILE_NAME = "data_topic_name_distrib_mistral_7b_4bit_type_new.csv"
RES_FILE = DATA_DIR.joinpath(RES_FILE_NAME)
# Remove old file
if os.path.exists(RES_FILE):
    os.remove(RES_FILE)
for i in tqdm(range(len(data_text))):
# for i in tqdm(range(20)):
    torch.cuda.empty_cache()
    SAMPLE = data_text[i]
    res_model = few_shot_pipeline(
        INSTRUCTION,
        SAMPLE,
        model,
        device="cuda",
        temp=0.3,
        n_token=100
    )
    # print(res_model)
    with open(RES_FILE, 'a') as file:
        file.write(f"{SAMPLE};{res_model}\n")


100%|██████████| 2363/2363 [1:23:16<00:00,  2.11s/it]
